In [1]:
import tensorflow as tf

import keras
from keras.models import load_model

import numpy as np
import cv2
import matplotlib.pyplot as plt

import scipy

Using TensorFlow backend.


In [2]:
# helpers vars for the notebook
already_trained = False
train_test_model = True

In [24]:
# The following code is just copied over from the baseline_cnn_notebook
# (with some modification)

class ModelParameters:

    def __init__(self,
                 training_data_path,
                 num_classes=28,
                 num_epochs=5,
                 batch_size=16,
                 image_rows=1708,
                 image_cols=1708,
                 row_scale_factor=4,
                 col_scale_factor=4,
                 n_channels=3,
                 shuffle=False):
        
        self.training_data_path = training_data_path
        self.num_classes = num_classes
        # what does n_epochs mean? it seems we pass this into the "epochs"
        # parameter on the "fit_generator" method on our keras model
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.row_dimension = np.int(image_rows / row_scale_factor)
        self.col_dimension = np.int(image_cols / col_scale_factor)
        self.n_channels = n_channels
        self.shuffle = shuffle


class ImagePreprocessor:

    def __init__(self, modelParameters):
        self.image_path = modelParameters.training_data_path
        self.n_channels = modelParameters.n_channels
        self.row_dimension = modelParameters.row_dimension
        self.col_dimension = modelParameters.col_dimension

    def preprocess(self, image):
        image = cv2.resize(image, (self.row_dimension, self.col_dimension))
        image = np.true_divide(image, 255)
        return image

    def load_image(self, image_id):
        return cv2.imread(self.image_path + image_id + '.jpg')

class ImageBatchGenerator(keras.utils.Sequence):
    
    def __init__(self, image_ids, dataframe, model_params, image_processor):
        '''
        Writing a child implementation of keras.utils.Sequence will help us
        manage our batches of data.
        Each sequence must implement __len__ and __getitem__
        This structure guarantees that the network will only train once on each
        sample per epoch which is not the case with generators.
        
        We can use this class to instantiate training and validation generators
        that we can pass into our keras model like:
        
        training_generator = ImageBatchLoader(...)
        validation_generator = ImageBatchLoader(...)
        model.set_generators(training_generator, validation_generator)
        '''
        self.image_ids = image_ids
        self.dataframe = dataframe

        # Helper classes
        self._imageProcessor = image_processor

        # Training parameters
        self.batch_size = model_params.batch_size
        self.dimensions = (model_params.row_dimension, model_params.col_dimension)
        self.n_channels = model_params.n_channels
        self.shuffle = model_params.shuffle

        # Run on_epoch_end in _init_ to init our first image batch
        self.on_epoch_end()
        
    def on_epoch_end(self):
        '''
        Tensorflow will run this method at the end of each epoch
        So this is where we will modify our batch.
        '''
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        # Denotes the number of batchs per epoch
        return int(np.floor(len(self.image_ids) / self.batch_size))
            
    def __getitem__(self, index):
        # Get this batches indexes
        indexes = self.indexes[index * self.batch_size:(index+1) * self.batch_size]

        # Get cooresponding image Ids
        batch_image_ids = [self.image_ids[i] for i in indexes]

        # Generate one batch of data
        X, y = self.__generator(batch_image_ids)
        
        return X, y
    
    def __generator(self, batch_image_ids):

        def get_target_class(imageid):
            # .loc will lookup the row where the passed in statement is true
            target = self.dataframe.loc[self.dataframe.imageId == imageid]
            target = target.lesion.values[0]
            return target

        X = np.empty((self.batch_size, *self.dimensions, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        z = np.empty((self.batch_size), dtype=int)
        

        for index, imageid in enumerate(batch_image_ids):
            image = self._imageProcessor.load_image(imageid)
            image = self._imageProcessor.preprocess(image)

            X[index] = image
            y[index] = get_target_class(imageid)
            z[index] = get_target_class(imageid)

        return X, { "classificaton_output": y, "attention_map_output": y }


class PredictGenerator:

    def __init__(self, image_ids, image_processor, image_path):
        self.image_processor = image_processor
        self.image_processor.image_path = image_path
        self.image_ids = image_ids

    def predict(self, model):
        
        y = np.empty(shape=(len(self.image_ids)))
        
        for n in range(len(self.image_ids)):
            image = self.image_processor.load_image(self.image_ids[n])
            image = self.image_processor.preprocess(image)
            image = image.reshape((1, *image.shape))
            y[n] = model.predict(image)
        
        return { "classificaton_output": y, "attention_map_output": y }

In [25]:
# Get the data

import pandas as pd

data = pd.read_csv("./lesion-csv.csv")
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

# All we care about at this point is the id and class
train_df = train_df.drop(
    ["Unnamed: 0", "Unnamed: 0.1", "teethNumbers", "description", "numberOfCanals", "date", "sequenceNumber"], axis=1)

test_df = test_df.drop(
    ["Unnamed: 0", "Unnamed: 0.1", "teethNumbers", "description", "numberOfCanals", "date", "sequenceNumber"], axis=1)

partition = {
    "train": train_df.imageId.values,
    "validation": test_df.imageId.values,
}

In [34]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import SGD

class SimpleModel:
    
    def __init__(self, model_params):
        self.img_rows = model_params.row_dimension
        self.img_cols = model_params.col_dimension
        self.n_channels = model_params.n_channels
        self.input_shape = (self.img_rows, self.img_cols, self.n_channels)
        self.num_epochs = model_params.num_epochs
        self.metrics = ['accuracy']

    @staticmethod
    def build_classification_branch(inputs):
        x = Conv2D(32, kernel_size=3, padding='same')(inputs)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(64, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = Conv2D(64, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(128, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = Conv2D(128, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(256)(x)
        x = Activation('relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation('sigmoid', name="classificaton_output")(x)
        
        return x
    
    @staticmethod
    def build_attention_map_branch(inputs):
        x = Conv2D(16, kernel_size=3, padding='same')(inputs)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Conv2D(32, kernel_size=3, padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=-1)(x)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.25)(x)
        
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation('relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(1)(x)
        x = Activation('sigmoid', name="attention_map_output")(x)
        
        return x
        
    def build_multi_output(self):
        input_shape = (self.img_rows, self.img_cols, self.n_channels)
        
        inputs = Input(shape=input_shape)
        cl_branch = SimpleModel.build_classification_branch(inputs)
        am_branch = SimpleModel.build_attention_map_branch(inputs)
        
        self.model = Model(
            inputs=inputs,
            outputs=[cl_branch, am_branch],
            name='simple_model_test')
        
    def build_model(self):
        input_1 = Input(shape=(224,224,3))
        
        conv_1 = Conv2D(64, kernel_size=3, activation='relu')
        glap_1 = GlobalAveragePooling2D()
        otpt_1 = Dense(1, activation="sigmoid")
        
        conv_2 = Conv2D(32, kernel_size=3, activation='relu')
        glap_2 = GlobalAveragePooling2D()
        otpt_2 = Dense(1, activation="sigmoid")
        
        
        conv_1_otpt = conv_1(input_1)
        glap_1_otpt = glap_1(conv_1_otpt)
        otpt_1_otpt = otpt_1(glap_1_otpt)
        
        conv_2_otpt = conv_2(conv_1_otpt)
        glap_2_otpt = glap_2(conv_2_otpt)
        otpt_2_otpt = otpt_2(glap_2_otpt)
        
        
        self.model = Model(inputs=input_1, outputs=[otpt_1.output])
    
    def compile_model(self):
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
        
        losses = {
            "classificaton_output": "binary_crossentropy", # TODO: change this to equation 5 fro mthe GAIN paper
            "attention_map_output": "binary_crossentropy"
        }
        losses_weights = {
            "classificaton_output": 1.0,
            "attention_map_output": 1.0
        }
        
        self.model.compile(
            loss=losses, loss_weights=losses_weights, optimizer=sgd, metrics=self.metrics)
    
    def set_generators(self, train_generator, validation_generator):
        self.training_generator = train_generator
        self.validation_generator = validation_generator
        
    def learn(self):
        return self.model.fit_generator(
            generator=self.training_generator,
            validation_data=self.validation_generator,
            epochs=self.num_epochs,
            use_multiprocessing=True,
            workers=8,
            verbose=1)

    def score(self):
        return self.model.evaluate_generator(
            generator=self.validation_generator,
            use_multiprocessing=True,
            workers=8)

    def predict(self, predict_generator):
        y = predict_generator.predict(self.model)
        return y

    def save(self, modeloutputpath):
        self.model.save(modeloutputpath)

    def load(self, modelinputpath):
        self.model = load_model(modelinputpath)

In [35]:
train_path = './lesion_images/all_images_processed/'

model_params = ModelParameters(
    train_path,
    num_epochs= 1 if train_test_model else 15,
    batch_size=16,
    image_rows=224,
    image_cols=224,
    row_scale_factor=1,
    col_scale_factor=1)

image_processor = ImagePreprocessor(model_params)

training_generator = ImageBatchGenerator(partition['train'], data, model_params, image_processor)
validation_generator = ImageBatchGenerator(partition['validation'], data, model_params, image_processor)
predict_generator = PredictGenerator(partition['validation'], image_processor, train_path)

In [36]:
simple_model = SimpleModel(model_params)
simple_model.build_multi_output()
simple_model.compile_model()
simple_model.set_generators(training_generator, validation_generator)

ValueError: Input 0 is incompatible with layer conv2d_10: expected ndim=4, found ndim=2

In [23]:
history = simple_model.learn()

Epoch 1/1
70/70 [==============================] - 8s 112ms/step - loss: 0.6962 - acc: 0.4964 - val_loss: 0.6948 - val_acc: 0.4875
